# Data Cleaning: Job Openings, Turnover, and Occupational Requirements

In [1]:
import os
import pandas as pd
import numpy as np
import missingno as msno

# See all rows and columns in the notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Section 1: Job Openings & Labor Turnover Survey by State

Source: U.S. Bureau of Labor Statistics (BLS)

Criteria: 
- Industries: Total Nonfarm
- Region: Total US (50 states + D.C.)
- Broken out by: State
- Data Elements: 1) Job openings, 2) Total Separations, and 3) Layoffs & Discharges (Note: This is a part of the Total Separations metric)
- Metrics By: Level in thousands

In [2]:
# Load datasets

# Define path to folder with raw data
folder_path = "../../Data/2024/BLS_job_openings_turnover"

# Initialize empty lists for each future dataset
list_openings = []
list_layoffs = []
list_total_separations = []

# Loop through each file
for file in os.listdir(folder_path):

    # Locate each .xlsx file and define the file path
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
    
        # Use the name of the file to name the dataframes
        parts_of_name = file.replace(".xlsx", "").split("-")
        state = parts_of_name[0]
        metric = "-".join(parts_of_name[1:])

        # Read each .xlsx file
        df = pd.read_excel(file_path)

        # Add a "state" col to the datasets
        df["State"] = state

        # Add each dataframe to the relevant lists
        if metric == "openings":
            list_openings.append(df)
        elif metric == "layoffs":
            list_layoffs.append(df)
        elif metric == "total-separation":
            list_total_separations.append(df)
        else: 
            print("Found an unexpected dataset in the raw, local folder. Please identify and try again.")

/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook con

In [3]:
# Preview one of the datasets
list_openings[2]

,Job Openings and Labor Turnover Survey,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,State
0,Original Data Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
2,Series Id:,JTS000000260000000JOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
3,Seasonally adjusted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
4,Industry:,Total nonfarm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
5,State/Region:,Michigan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
6,Area:,All areas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
7,Data Element:,Job openings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
8,Size Class:,All size classes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan
9,Rate/Level:,Level - In Thousands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,michigan


In [4]:
# List of lists
l = [list_openings, list_layoffs, list_total_separations]

# Loop through each list
for df_list in l:

    # Loop through each dataset
    for i, df in enumerate(df_list):
        
        # Extract state
        state = df["State"].iloc[0]

        # Change the header
        col_head = df.iloc[12]

        # Remove irrelevant rows
        df_clean = df.iloc[13:].reset_index(drop = True)

        # Apply extracted header to cleaned df
        df_clean.columns = col_head

        # Insert state col back
        df_clean.insert(0, "state", state)

        # Save changes back to the original list
        df_list[i] = df_clean

In [5]:
# Create the three needed dataset
openings = pd.concat(list_openings, ignore_index = True)
layoffs = pd.concat(list_layoffs, ignore_index = True)
total_separations = pd.concat(list_total_separations, ignore_index = True)

# Get rid of irrelevant cols
openings = openings.iloc[:, :14] 
layoffs = layoffs.iloc[:, :14] 
total_separations = total_separations.iloc[:, :14] 

In [6]:
# Multiply numerical cols by 1000

job_df_list = [openings, layoffs, total_separations]

for df in job_df_list:
    df[["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]] = df[["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]].mul(1000)

In [7]:
# Clean up "state" col
states_list = {"alabama": "al", "alaska": "ak", "arizona": "az", "arkansas": "ar", 
            "california": "ca", "colorado": "co", "connecticut": "ct", "delaware": "de", 
            "florida": "fl", "georgia": "ga", "hawaii": "hi", "idaho": "id", "illinois": "il", 
            "indiana": "in", "iowa": "ia", "kansas": "ks", "kentucky": "ky", "louisiana": "la", 
            "maine": "me", "maryland": "md", "massachusetts": "ma", "michigan": "mi", "minnesota": "mn", 
            "mississippi": "ms", "missouri": "mo", "montana": "mt", "nebraska": "ne", "nevada": "nv", "newhampshire": "nh", 
            "newjersey": "nj", "newmexico": "nm", "newyork": "ny", "northcarolina": "nc", "northdakota": "nd", "ohio": "oh", 
            "oklahoma": "ok", "oregon": "or", "pennsylvania": "pa", "rhodeisland": "ri", "southcarolina": "sc", "southdakota": "sd", 
            "tennessee": "tn", "texas": "tx", "utah": "ut", "vermont": "vt", "virginia": "va", 
            "washington": "wa", "west virginia": "wv", "wisconsin": "wi", "wyoming": "wy", "dc": "dc"
}

for df in job_df_list:
    for full_name, abbrv in states_list.items():
        find_state = df["state"].astype(str).str.contains(full_name, na = False)
        df.loc[find_state, "state"] = abbrv

In [8]:
layoffs.head(20)

12,state,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,nh,2018,11000,9000,18000,7000,8000,11000,12000,9000,10000,9000,9000,9000
1,nh,2019,8000,7000,9000,9000,10000,9000,8000,9000,9000,10000,10000,10000
2,nh,2020,9000,10000,64000,47000,12000,10000,8000,6000,6000,13000,9000,9000
3,nh,2021,9000,9000,7000,7000,7000,5000,8000,9000,7000,7000,11000,7000
4,nh,2022,9000,7000,8000,7000,8000,8000,8000,9000,9000,16000,9000,9000
5,nh,2023,15000,9000,10000,8000,8000,7000,9000,7000,6000,12000,8000,8000
6,nh,2024,10000,11000,9000,10000,10000,11000,14000,8000,9000,6000,8000,7000
7,nh,2025,8000,10000,8000,9000,9000,9000,11000,12000,NaN,NaN,NaN,NaN
8,me,2018,11000,7000,9000,7000,8000,19000,19000,11000,9000,10000,8000,9000
9,me,2019,9000,9000,9000,9000,9000,10000,7000,8000,8000,11000,10000,8000


In [9]:
print(f"Openings: {openings.isnull().any()}")
print(f"Layoffs & Discharges: {layoffs.isnull().any()}")
print(f"Total Separations: {total_separations.isnull().any()}")

Openings: 12
state    False
Year     False
Jan      False
Feb      False
Mar      False
Apr      False
May      False
Jun      False
Jul      False
Aug      False
Sep       True
Oct       True
Nov       True
Dec       True
dtype: bool
Layoffs & Discharges: 12
state    False
Year     False
Jan      False
Feb      False
Mar      False
Apr      False
May      False
Jun      False
Jul      False
Aug      False
Sep       True
Oct       True
Nov       True
Dec       True
dtype: bool
Total Separations: 12
state    False
Year     False
Jan      False
Feb      False
Mar      False
Apr      False
May      False
Jun      False
Jul      False
Aug      False
Sep       True
Oct       True
Nov       True
Dec       True
dtype: bool


In [10]:
# Export cleaned dataset
openings.to_csv("../Cleaned data/bls/cleaned_job_openings.csv", index=False)
layoffs.to_csv("../Cleaned data/bls/cleaned_job_layoffs.csv", index=False)
total_separations.to_csv("../Cleaned data/bls/cleaned_job_total_separations.csv", index=False)

## Section 2: Job Openings & Labor Turnover Survey by Sector

Source: U.S. Bureau of Labor Statistics (BLS)

Criteria: 
- Industries: Total Nonfarm
- Region: Total US 
- Broken out by: Sector
- Data Elements: 1) Job openings, 2) Total Separations, and 3) Layoffs & Discharges (Note: This is a part of the Total Separations metric)
- Metrics By: Level in thousands

In [11]:
# Load datasets

# Define path to folder with raw data
sector_folder_path = "../../Data/2024/bls_job_openings_by_sector"

# Initialize empty lists for each future dataset
l_openings = []
l_layoffs = []
l_total_separations = []

# Loop through each file
for file in os.listdir(sector_folder_path):

    # Locate each .xlsx file and define the file path
    if file.endswith(".xlsx"):
        file_path = os.path.join(sector_folder_path, file)
    
        # Use the name of the file to name the dataframes
        parts_of_name = file.replace(".xlsx", "").split("_")
        sector = parts_of_name[0]
        metric = "-".join(parts_of_name[1:])

        # Read each .xlsx file
        df = pd.read_excel(file_path)

        # Add a "state" col to the datasets
        df["Sector"] = sector

        # Add each dataframe to the relevant lists
        if metric == "openings":
            l_openings.append(df)
        elif metric == "layoffs":
            l_layoffs.append(df)
        elif metric == "total-separation":
            l_total_separations.append(df)
        else: 
            print("Found an unexpected dataset in the raw, local folder. Please identify and try again.")

/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook con

In [13]:
#Preview a dataset
l_openings[6]

,Job Openings and Labor Turnover Survey,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Sector
0,Original Data Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
2,Series Id:,JTS540099000000000JOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
3,Seasonally adjusted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
4,Industry:,Professional and business services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
5,State/Region:,Total US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
6,Area:,All areas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
7,Data Element:,Job openings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
8,Size Class:,All size classes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services
9,Rate/Level:,Level - In Thousands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,professional-business-services


In [14]:
# List of lists
ls = [l_openings, l_layoffs, l_total_separations]

# Loop through each list
for df_list in ls:

    # Loop through each dataset
    for i, df in enumerate(df_list):
        
        # Extract sector
        sector = df["Sector"].iloc[0]

        # Change the header
        col_head = df.iloc[12]

        # Remove irrelevant rows
        df_clean = df.iloc[13:].reset_index(drop = True)

        # Apply extracted header to cleaned df
        df_clean.columns = col_head

        # Insert state col back
        df_clean.insert(0, "sector", sector)

        # Save changes back to the original list
        df_list[i] = df_clean

In [15]:
# Create the three needed dataset
sector_openings = pd.concat(l_openings, ignore_index = True)
sector_layoffs = pd.concat(l_layoffs, ignore_index = True)
sector_total_separations = pd.concat(l_total_separations, ignore_index = True)

# Get rid of irrelevant cols
sector_openings = sector_openings.iloc[:, :14] 
sector_layoffs = sector_layoffs.iloc[:, :14] 
sector_total_separations = sector_total_separations.iloc[:, :14] 

In [17]:
# Clean "sector" col 
job_df_list = [sector_openings, sector_layoffs, sector_total_separations]

for df in job_df_list:
    df["sector"] = df["sector"].astype(str).str.replace("-", " ")

for df in job_df_list:
    df[["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]] = df[["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]].mul(1000)

In [18]:
sector_openings.head()

12,sector,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,state and local government other,2015,283000,271000,275000,281000,289000,210000,282000,253000,270000,257000,263000,270000
1,state and local government other,2016,260000,240000,251000,262000,286000,266000,288000,270000,282000,280000,334000,258000
2,state and local government other,2017,261000,287000,280000,287000,270000,324000,284000,295000,310000,300000,294000,318000
3,state and local government other,2018,301000,325000,334000,325000,319000,337000,365000,373000,320000,295000,336000,320000
4,state and local government other,2019,363000,355000,360000,363000,369000,403000,316000,398000,391000,413000,410000,423000


In [19]:
print(f"Openings: {sector_openings.isnull().any()}")
print(f"Layoffs & Discharges: {sector_layoffs.isnull().any()}")
print(f"Total Separations: {sector_total_separations.isnull().any()}")

Openings: 12
sector    False
Year      False
Jan       False
Feb       False
Mar       False
Apr       False
May       False
Jun       False
Jul       False
Aug       False
Sep        True
Oct        True
Nov        True
Dec        True
dtype: bool
Layoffs & Discharges: 12
sector    False
Year      False
Jan       False
Feb       False
Mar       False
Apr       False
May       False
Jun       False
Jul       False
Aug       False
Sep        True
Oct        True
Nov        True
Dec        True
dtype: bool
Total Separations: 12
sector    False
Year      False
Jan       False
Feb       False
Mar       False
Apr       False
May       False
Jun       False
Jul       False
Aug       False
Sep        True
Oct        True
Nov        True
Dec        True
dtype: bool


In [20]:
# Export cleaned dataset
sector_openings.to_csv("../Cleaned data/bls/cleaned_job_openings_by_sector.csv", index=False)
sector_layoffs.to_csv("../Cleaned data/bls/cleaned_job_layoffs_by_sector.csv", index=False)
sector_total_separations.to_csv("../Cleaned data/bls/cleaned_job_total_separations_by_sector.csv", index=False)

## Section 3: Occupational Requirements Survey